In [1]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [2]:
%cd /content

/content


In [3]:
!unzip /drive/MyDrive/projects/sign_language_recognition/ICS504-HW3.zip

Streaming output truncated to the last 5000 lines.
  inflating: ICS471-HW4/train/0009/02_0009_(27_02_21_20_43_02)_c/02_0009_(27_02_21_20_43_02)_c_0023.jpg  
  inflating: ICS471-HW4/train/0009/02_0009_(27_02_21_20_43_02)_c/02_0009_(27_02_21_20_43_02)_c_0024.jpg  
  inflating: ICS471-HW4/train/0009/02_0009_(27_02_21_20_43_02)_c/02_0009_(27_02_21_20_43_02)_c_0025.jpg  
  inflating: ICS471-HW4/train/0009/02_0009_(27_02_21_20_43_02)_c/02_0009_(27_02_21_20_43_02)_c_0026.jpg  
  inflating: ICS471-HW4/train/0009/02_0009_(27_02_21_20_43_02)_c/02_0009_(27_02_21_20_43_02)_c_0027.jpg  
  inflating: ICS471-HW4/train/0009/02_0009_(27_02_21_20_43_02)_c/02_0009_(27_02_21_20_43_02)_c_0028.jpg  
  inflating: ICS471-HW4/train/0009/02_0009_(27_02_21_20_43_02)_c/02_0009_(27_02_21_20_43_02)_c_0029.jpg  
  inflating: ICS471-HW4/train/0009/02_0009_(27_02_21_20_43_02)_c/02_0009_(27_02_21_20_43_02)_c_0030.jpg  
  inflating: ICS471-HW4/train/0009/02_0009_(27_02_21_20_43_02)_c/02_0009_(27_02_21_20_43_02)_c_0031.j

In [4]:
!pip install keras_nlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.8/466.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 20.1 MB/s eta 0:00:00


In [5]:
import keras
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from tensorflow.keras import backend as K
import keras_nlp
import numpy as np
import os
import random
import cv2
from PIL import Image
import glob
from collections import Counter
from sklearn.model_selection import train_test_split
from joblib import load, dump

tf.config.run_functions_eagerly(True)

In [26]:
save_dir = '/drive/MyDrive/projects/sign_language_recognition/saved_weights_updated'
os.makedirs(save_dir, exist_ok=True)

In [7]:
filenames = np.asarray(glob.glob('./ICS471-HW4/train/*/*'))
labels = np.asarray([int(file.split('/')[-2])-1 for file in filenames])

print(len(filenames), len(labels))
print(Counter(labels))

534 534
Counter({6: 70, 7: 62, 4: 60, 9: 51, 1: 50, 5: 49, 8: 48, 3: 48, 0: 48, 2: 48})


In [8]:
file_train, file_val, label_train, label_val = train_test_split(filenames, labels, test_size=0.1, random_state=42)

print(file_train.shape, label_train.shape)
print(file_val.shape, label_val.shape)

(480,) (480,)
(54,) (54,)


In [9]:
with open('./ICS471-HW4/groundTruth.txt','r') as f:
    data = f.read()

In [27]:
texts = []
for cap in data.split('\n'):
    caption = "<bos> " + cap + " <eos>"
    texts.append(caption)

print(texts)

['<bos> اسم الله <eos>', '<bos> الحمد الله <eos>', '<bos> جميع الصم العرب  السامع <eos>', '<bos> السلام عليكم رحمة الله بركة <eos>', '<bos> اليوم اقدم انتم برنامج اخر <eos>', '<bos> موضوع دراسة لغة الاشارة العربية <eos>', '<bos> كلمات اليوم متفرقة في الدين <eos>', '<bos> ايضا كلمات عادية <eos>', '<bos> لا شرك الله <eos>', '<bos> الله اكبر <eos>']


In [28]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index
index_to_word = {v: k for k, v in word_index.items()}
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, padding='post', truncating='post', maxlen=max_length)
vocab_size = len(word_index) + 1

dump(tokenizer, save_dir+'/tokenizer.joblib')

['/drive/MyDrive/projects/sign_language_recognition/saved_weights_updated/tokenizer.joblib']

In [12]:
print('max_length:', max_length)
print('Vocab size:', vocab_size)

max_length: 7
Vocab size: 33


In [13]:
HEIGHT = 128
WIDTH = 128
FRAMES = 40
embedding_size = 16
hidden_size = 20
epochs = 50
batch_size = 1

In [14]:
class Conv2Plus1D(tf.keras.layers.Layer):
  def __init__(self, filters, kernel_size, padding):
    """
      A sequence of convolutional layers that first apply the convolution operation over the
      spatial dimensions, and then the temporal dimension. 
    """
    super().__init__()
    self.seq = tf.keras.Sequential([  
        # Spatial decomposition
        layers.Conv3D(filters=filters,
                      kernel_size=(1, kernel_size[1], kernel_size[2]),
                      padding=padding),
        # Temporal decomposition
        layers.Conv3D(filters=filters, 
                      kernel_size=(kernel_size[0], 1, 1),
                      padding=padding)
        ])

  def call(self, x):
    return self.seq(x)

In [15]:
class ResidualMain(tf.keras.layers.Layer):
  """
    Residual block of the model with convolution, layer normalization, and the
    activation function, ReLU.
  """
  def __init__(self, filters, kernel_size):
    super().__init__()
    self.seq = tf.keras.Sequential([
        Conv2Plus1D(filters=filters,
                    kernel_size=kernel_size,
                    padding='same'),
        layers.LayerNormalization(),
        layers.ReLU(),
        Conv2Plus1D(filters=filters, 
                    kernel_size=kernel_size,
                    padding='same'),
        layers.LayerNormalization()
    ])

  def call(self, x):
    return self.seq(x)

In [16]:
class Project(tf.keras.layers.Layer):
  """
    Project certain dimensions of the tensor as the data is passed through different 
    sized filters and downsampled. 
  """
  def __init__(self, units):
    super().__init__()
    self.seq = tf.keras.Sequential([
        layers.Dense(units),
        layers.LayerNormalization()
    ])

  def call(self, x):
    return self.seq(x)

In [17]:
def add_residual_block(input, filters, kernel_size):
  """
    Add residual blocks to the model. If the last dimensions of the input data
    and filter size does not match, project it such that last dimension matches.
  """
  out = ResidualMain(filters, 
                     kernel_size)(input)

  res = input
  # Using the Keras functional APIs, project the last dimension of the tensor to
  # match the new filter size
  if out.shape[-1] != input.shape[-1]:
    res = Project(out.shape[-1])(res)

  return layers.add([res, out])

In [18]:
class attention(tf.keras.layers.Layer):
    
    def __init__(self, return_sequences=True,**kwargs):
        self.return_sequences = return_sequences
        super(attention,self).__init__()

    def get_config(self):
      config = super().get_config().copy()
      config.update({
          'return_sequences': self.return_sequences 
      })
      return config
        
    def build(self, input_shape):
        
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),
                               initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1),
                               initializer="zeros")
        
        super(attention,self).build(input_shape)
        
    def call(self, x):
        
        e = K.tanh(K.dot(x,self.W)+self.b)
        a = K.softmax(e, axis=1)
        output = x*a
        
        if self.return_sequences:
            return output
        
        return K.sum(output, axis=1)

In [19]:
def wer(y_true, y_pred):
    """
    Computes the edit distance between two sequences.
    """
    dist = keras_nlp.metrics.EditDistance()
    if not isinstance(y_true, np.ndarray):
        y_true = y_true.numpy()
    if not isinstance(y_pred, np.ndarray):
        y_pred = y_pred.numpy()

    score = 0
    for i in range(len(y_true)):
        idx = np.argmax(y_pred[i])
        pred = np.zeros_like(y_pred[i])
        pred[idx] = 1.0
        print(y_true[i].shape, pred.shape)
        print(dist(y_true[i], pred).numpy())
        score += dist(y_true[i], pred).numpy()
        
    return score/len(y_true)

In [58]:
video_input = layers.Input(shape=(FRAMES, HEIGHT, WIDTH, 3))
x = Conv2Plus1D(filters=16, kernel_size=(3, 7, 7), padding='same')(video_input)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.MaxPooling3D((1, 2, 2))(x)
# x = layers.Dropout(0.3)(x)

# Block 1
x = add_residual_block(x, 32, (3, 3, 3))
x = layers.MaxPooling3D((1, 2, 2))(x)
# x = layers.Dropout(0.3)(x)

# Block 2
x = add_residual_block(x, 64, (3, 3, 3))
x = layers.MaxPooling3D((1, 2, 2))(x)
# x = layers.Dropout(0.3)(x)

# Block 3
x = add_residual_block(x, 128, (3, 3, 3))
x = layers.MaxPooling3D((1, 2, 2))(x)
# x = layers.Dropout(0.3)(x)

# Block 4
x = add_residual_block(x, 256, (3, 3, 3))
# x = layers.Dropout(0.3)(x)
encoder_output = layers.TimeDistributed(layers.GlobalAveragePooling2D())(x)

In [59]:
encoder = layers.Bidirectional(tf.keras.layers.LSTM(hidden_size, return_state=True, return_sequences=True, name='encoder_lstm1'))
encoder_seq_output, state_h, state_c, state_d, state_e = encoder(encoder_output)
attention_output = attention(return_sequences=True, name='attention')(encoder_seq_output)
encoder = layers.Bidirectional(tf.keras.layers.LSTM(hidden_size, return_state=True, return_sequences=True, name='encoder_lstm2'))
_, state_h_1, state_c_1, state_d_1, state_e_1 = encoder(attention_output)
encoder_states = [state_h_1,state_c_1, state_d_1, state_e_1]

decoder_inputs = tf.keras.layers.Input(shape=(max_length-1,), name="decoder_inputs")
decoder_embedding = layers.Embedding(input_dim=vocab_size,output_dim=4,input_length=max_length, mask_zero=True)(decoder_inputs)

decoder_lstm = layers.Bidirectional(tf.keras.layers.LSTM(hidden_size, name='decoder_lstm'))
decoder_outputs = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_outputs = tf.keras.layers.Dropout(0.3)(decoder_outputs)
decoder_dense = tf.keras.layers.Dense(vocab_size, activation='softmax', name='decoder_sigmoid')
decoder_outputs = decoder_dense(decoder_outputs)

model = tf.keras.models.Model([video_input, decoder_inputs], decoder_outputs)
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 40, 128, 12  0           []                               
                                8, 3)]                                                            
                                                                                                  
 conv2_plus1d_9 (Conv2Plus1D)   (None, 40, 128, 128  3152        ['input_2[0][0]']                
                                , 16)                                                             
                                                                                                  
 batch_normalization_1 (BatchNo  (None, 40, 128, 128  64         ['conv2_plus1d_9[0][0]']         
 rmalization)                   , 16)                                                       

In [60]:
def frames_from_video_file(path, n_frames, dim):
    image_paths = sorted(glob.glob(path+'/*.*g'))
    interval = len(image_paths)//n_frames
    frames = []
    for i in range(len(image_paths)):
        if i % interval == 0:
            img = np.asarray(Image.open(image_paths[i]).convert('RGB').resize(dim))/255.0
            frame = tf.image.convert_image_dtype(img, tf.float32)
            frames.append(frame)
        if len(frames) == n_frames:
            break

    return np.asarray(frames)

In [61]:
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=batch_size, dim=(WIDTH,HEIGHT), n_channels=3, n_frames=FRAMES, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.n_frames = n_frames
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)  

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X, decoder_input_data, decoder_target_data = [], [], []

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            frames = frames_from_video_file(ID, self.n_frames, self.dim)

            # Store class
            label = int(ID.split('/')[-2])-1
            seq = padded_sequences[label]

            for j in range(1, max_length):
                in_seq, out_seq = seq[:j], seq[j]
                in_seq = pad_sequences([in_seq], padding='post', truncating='post', maxlen=max_length-1)[0]
                out_seq = keras.utils.to_categorical([out_seq], num_classes=vocab_size)[0]

                X.append(frames)
                decoder_input_data.append(in_seq)
                decoder_target_data.append(out_seq)

        X = np.asarray(X)
        decoder_input_data = np.asarray(decoder_input_data)
        decoder_target_data = np.asarray(decoder_target_data)

        return [X, decoder_input_data], decoder_target_data

In [62]:
train_gen = DataGenerator(file_train, label_train)
val_gen = DataGenerator(file_val, label_val)

validation_steps = len(val_gen)
steps_per_epoch = len(train_gen)

In [63]:
# x, y = val_gen.__getitem__(0)

In [64]:
# prepare model saving directory.
model_name = 'best_weight_{epoch}.h5'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

In [65]:
# reduce learning rate by sqrt(0.1) if the loss does not improve in 5 epochs
lr_reducer = ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.2, 
                              patience=3, 
                              verbose=1, 
                              min_delta=0.0001
                             )

# save weights for future use (e.g. reload parameters w/o training)
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=False,
                             save_weights_only=True)

filename = filepath+'.csv'
csv_log = CSVLogger(filename, separator=',', append=False)

# called every epoch
callbacks = [lr_reducer, checkpoint, csv_log]

In [ ]:
history = model.fit(
    train_gen, steps_per_epoch=steps_per_epoch,
    validation_data=val_gen, validation_steps=validation_steps,
    epochs=epochs, callbacks=callbacks)

Epoch 1/50
480/480 [==============================] - ETA: 0s - loss: 3.2516 - accuracy: 0.2191
Epoch 1: saving model to /drive/MyDrive/projects/sign_language_recognition/saved_weights_updated/best_weight_1.h5
480/480 [==============================] - 702s 1s/step - loss: 3.2516 - accuracy: 0.2191 - val_loss: 2.8673 - val_accuracy: 0.2562 - lr: 1.0000e-04
Epoch 2/50
480/480 [==============================] - ETA: 0s - loss: 2.8787 - accuracy: 0.2083
Epoch 2: saving model to /drive/MyDrive/projects/sign_language_recognition/saved_weights_updated/best_weight_2.h5
480/480 [==============================] - 698s 1s/step - loss: 2.8787 - accuracy: 0.2083 - val_loss: 2.7104 - val_accuracy: 0.2562 - lr: 1.0000e-04
Epoch 3/50
480/480 [==============================] - ETA: 0s - loss: 2.7646 - accuracy: 0.2250
Epoch 3: saving model to /drive/MyDrive/projects/sign_language_recognition/saved_weights_updated/best_weight_3.h5
480/480 [==============================] - 693s 1s/step - loss: 2.7646 -

In [41]:
model.load_weights(save_dir+'/best_weight_3.h5')

In [42]:
file_test = np.asarray(glob.glob('./ICS471-HW4/test/*/*'))
label_test = np.asarray([int(file.split('/')[-2])-1 for file in file_test])

print(len(file_test), len(label_test))
print(Counter(label_test))

66 66
Counter({4: 12, 8: 6, 3: 6, 0: 6, 1: 6, 5: 6, 6: 6, 7: 6, 2: 6, 9: 6})


In [43]:
test_gen = DataGenerator(file_test, label_test)

In [44]:
model.evaluate(test_gen)

66/66 [==============================] - 35s 528ms/step - loss: 2.8115 - accuracy: 0.2121


[2.81146240234375, 0.21212121844291687]

In [55]:
def predict_on_single_video(path):
    dims = (WIDTH,HEIGHT)
    frames = frames_from_video_file(path, FRAMES, dims)
    frames = np.expand_dims(frames,axis=0)
    in_text = ['<bos>']
    for j in range(1, max_length):
        seq = np.asarray(tokenizer.texts_to_sequences(in_text)).flatten()
        seq = np.expand_dims(seq, axis=0)
        in_seq = pad_sequences(seq, padding='post', truncating='post', maxlen=max_length-1)[0]
        in_seq = np.expand_dims(in_seq,axis=0)

        yhat = model.predict([frames,in_seq])
        idx = np.argmax(yhat)
        print(yhat)
        word = index_to_word[idx]
        in_text += [word]
        if word == 'eos':
            break
    
    final = in_text[1:-1]
    final = ' '.join(final)
    return final

In [57]:
predict_on_single_video(file_test[10])